# Basic Molecular Dynamics 

This notebook gives some structural information, and a 3D visualization without the need for any softwares like PyMol etc. 

It is mandatory to run the imports and load the pdb file input sections first! After this, it is possible to run any of the numbered sections listed below, independentantly. Simply click on the index links and run the cells under that section. 

-----------------------------------------------------------------------------

# Table of Contents
* [Mandatory imports](#Imports)
* [Load pdb file](#Load-input)


------------------------------------------
1. [Start and End residues](#Start-and-End-residues)
2. [Radius of gyration](#Radius-of-gyration)
3. [Salt Bridges](#Salt-bridges)
4. [3D View](#3D)
5. [Stability Simulation](#Stability-Simulations)
6. [Neighbors](#Neighbors)

------------------------------------------------

## Imports


In [1]:
import warnings
warnings.filterwarnings("ignore")

import MDAnalysis as mda
from MDAnalysis.lib import distances
import nglview as nv

import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline
plt.matplotlib.style.use("ggplot")

## Load input

In [2]:
u = mda.Universe('Data_files/GH26.pdb')

## Start and End residues

In [ ]:
n_term = u.select_atoms('segid A and name N')[0]
c_term = u.select_atoms('segid A and name C')[-1]
print("Starts at : ", n_term.resid, n_term.resname)
print("Ends at : ", c_term.resid, c_term.resname)

## Radius of gyration

In [ ]:
bb = u.select_atoms('protein and backbone')
r = c_term.position - n_term.position
d = np.linalg.norm(r)
rgyr = bb.radius_of_gyration()
print("End to end distance = %f Angstroem" %(d))
print("Radius of gyration = %f Angstroem" % (rgyr))

## Salt bridges

In [3]:
def get_contact_residues(acidic, basic, dmax=3.5):
    """Find all residues for which acidics are within dmax of basics."""

    dist = distances.distance_array(acidic.positions, basic.positions)
    exposed_atoms = np.any(dist <= dmax, axis=1)
    return acidic[exposed_atoms].residues

In [ ]:
from scipy import stats
print(stats.describe(salt_bridges))
print(stat.describe_salt(bridges))
this section describes what happens when salt bridges are established bewtee

In [ ]:
len(salt_bridges)

In [4]:
CA = u.select_atoms("protein and name CA") 

acidics = CA.select_atoms("resname ASP GLU")
print(acidics.n_residues, " acidic residues found")
basics = CA.select_atoms("resname LYS ARG")
print(basics.n_residues, " basic residues found")


43  acidic residues found
33  basic residues found


In [6]:
salt_bridges = get_contact_residues(acidics, basics, dmax=3.5) #Alter dmax value if required
list(salt_bridges)

[]

In [ ]:
from MDAnalysis.analysis import contacts
sel_acs = "(resname ARG LYS) and (name NH* NZ)"
sel_bcs = "(resname ASP GLU) and (name OE* OD*)"

acs = u1.select_atoms(sel_acs)
bcs = u1.select_atoms(sel_bcs)
cal1 = contacts.Contacts(u1, selection=(sel_acs, sel_bcs), refgroup=(acs, bcs), radius=6.0)
cal1.run()


In [ ]:
plt.plot(cal1.timeseries[:,0], cal1.timeseries[:,1])

## 3D

In [ ]:
w = nv.show_mdanalysis(u)
w

## Stability Simulations

P.S. This only works if you have a multi frame pdb file as your input.

The output is a graph which shows the radius of gyration in Angstroem unit for each frame. 
Interpretation is that the less change in Radius of gyration over time indicates more stability of the molecule,

In [ ]:
u1 = mda.Universe('Data_files/movie.pdb') #Input multi frame pdb file
bb1 = u1.select_atoms('protein and backbone')

results = np.zeros((bb1.atoms.universe.trajectory.n_frames, 2))
for i, ts in enumerate(bb1.atoms.universe.trajectory):
    rgyr = bb1.radius_of_gyration()
    results[i, :] = (ts.time, rgyr)

In [ ]:
plt.plot(results[:, 0], results[:, 1], label="Radius of gyration over time")
plt.xlabel("Time $t$ (ps)")
plt.ylabel("Radius of Gyration (Å)")
plt.legend(loc="best");

## Neighbors
Coming soon.....